In [4]:
import pandas as pd
import os

# Define the paths to the horse and race data files
horse_data_files = [f"C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_{year}.csv" for year in range(2019, 2021)]
race_data_files = [f"C:/Users/Elakkiya/Downloads/Horse Race Prediction/races_{year}.csv" for year in range(2019, 2021)]

horse_data_list = [pd.read_csv(file, low_memory=False) for file in horse_data_files]
race_data_list = [pd.read_csv(file, low_memory=False) for file in race_data_files]  


# Concatenate all years' data into single DataFrames
horse_data = pd.concat(horse_data_list, ignore_index=True)
race_data = pd.concat(race_data_list, ignore_index=True)


In [2]:
# Check for missing values
print(horse_data.isnull().sum())
print(race_data.isnull().sum())

rid                  0
horseName            0
age                  0
saddle             116
decimalPrice         0
isFav                0
trainerName         21
jockeyName           3
position             0
positionL        46446
dist             78428
weightSt             0
weightLb             0
overWeight      313278
outHandicap     316787
headGear        202202
RPR              91097
TR              165336
OR              139827
father               0
mother               1
gfather            206
runners              0
margin               0
weight               0
res_win              0
res_place            0
price           226351
dtype: int64
rid                0
course             0
time               0
date               0
title              0
rclass         14823
band           19556
ages               0
distance           0
condition          1
hurdles        24371
prizes             0
winningTime        0
prize              2
metric             0
countryCode        0
ncond  

In [5]:
# Define a function to drop columns with more than a specified percentage of null values
def drop_high_null_columns(df, threshold=0.7):
    null_percentage = df.isnull().sum() / len(df)
    columns_to_drop = null_percentage[null_percentage > threshold].index
    return df.drop(columns=columns_to_drop)

# Drop columns with more than 70% null values
horse_data_cleaned = drop_high_null_columns(horse_data, threshold=0.7)
race_data_cleaned = drop_high_null_columns(race_data, threshold=0.7)

In [24]:
print(horse_data_cleaned.isnull().sum())
print(race_data_cleaned.isnull().sum())

rid                  0
horseName            0
age                  0
saddle             116
decimalPrice         0
isFav                0
trainerName         21
jockeyName           3
position             0
positionL        46446
dist             78428
weightSt             0
weightLb             0
headGear        202202
RPR              91097
TR              165336
OR              139827
father               0
mother               1
gfather            206
runners              0
margin               0
weight               0
res_win              0
res_place            0
dtype: int64
rid                0
course             0
time               0
date               0
title              0
rclass         14823
band           19556
ages               0
distance           0
condition          1
prizes             0
winningTime        0
prize              2
metric             0
countryCode        0
ncond              0
class              0
dtype: int64


In [6]:
import pandas as pd
import re

# Define cleaning and normalization functions
def clean_distance(dist):
    if pd.isnull(dist):
        return None
    dist_str = str(dist).lower()
    miles_match = re.search(r'(\d+)m', dist_str)
    furlongs_match = re.search(r'(\d+)f', dist_str)
    miles = int(miles_match.group(1)) if miles_match else 0
    furlongs = 0
    if furlongs_match:
        furlongs = int(furlongs_match.group(1))
    fractional_match = re.search(r'(\d+)/(\d+)f', dist_str)
    if fractional_match:
        furlongs += int(fractional_match.group(1)) / int(fractional_match.group(2))
    return miles * 8 + furlongs

def normalize_time(time_str):
    if pd.isnull(time_str):
        return None
    time_parts = time_str.split(':')
    if len(time_parts) == 2:
        return int(time_parts[0]) * 60 + int(time_parts[1])
    elif len(time_parts) == 3:
        return int(time_parts[0]) * 3600 + int(time_parts[1]) * 60 + int(time_parts[2])
    return None

def clean_ages(ages):
    if pd.isnull(ages):
        return None
    match = re.search(r'\d+', str(ages))
    return int(match.group(0)) if match else None

def clean_title(title):
    return re.sub(r'[^a-zA-Z0-9\s]', '', title).strip() if title else title


def extract_numeric(value):
    if pd.isnull(value):
        return None
    match = re.search(r'\d+', str(value))
    return int(match.group(0)) if match else None


def clean_band(band):
    if pd.isnull(band) or band.strip() == '':
        return None  # Return None for empty or missing values

    band = str(band).strip()  # Remove leading/trailing whitespace

    # Handle ranges like '80-109', split and return the average of the range
    if '-' in band:
        band_range = band.split('-')

        # Check if both parts of the range are valid numbers
        if len(band_range) == 2 and band_range[0].strip() and band_range[1].strip():
            try:
                lower_bound = int(band_range[0].strip())
                upper_bound = int(band_range[1].strip())
                return (lower_bound + upper_bound) / 2  # Return the average
            except ValueError:
                return None  # If any part can't be converted to int, return None

    # If no range, return the numeric value if valid
    if band.isnumeric():
        return int(band)

    return None  # Return None if the value is not valid

def clean_rclass(rclass):
    if pd.isnull(rclass):
        return None
    match = re.search(r'\d+', str(rclass))
    return int(match.group(0)) if match else None


# Drop 'prizes' column in race data
if 'prizes' in race_data_cleaned.columns:
    race_data_cleaned.drop('prizes', axis=1, inplace=True)

# Apply cleaning functions if the 'rclass' column exists
if 'rclass' in horse_data_cleaned.columns:
    horse_data_cleaned['rclass'] = horse_data_cleaned['rclass'].apply(clean_rclass)

race_data_cleaned['distance'] = race_data_cleaned['distance'].apply(clean_distance)

race_data_cleaned['ages'] = race_data_cleaned['ages'].apply(clean_ages)

race_data_cleaned['band'] = race_data_cleaned['band'].apply(clean_band)

# Print cleaned DataFrames
print(horse_data_cleaned.head(10))
print(race_data_cleaned.head(10))



     rid      horseName  age  saddle  decimalPrice  isFav trainerName  \
0  11499         Picken  4.0     6.0      0.312500      0      J Size   
1  11499  Noble De Love  6.0     7.0      0.333333      1     F C Lor   
2  11499    Fresh Power  5.0     3.0      0.200000      0    Y S Tsui   
3  11499     Happy Hour  4.0     5.0      0.017544      0     C H Yip   
4  11499      Super Fun  5.0     8.0      0.090909      0        L Ho   
5  11499  Blizzing Away  3.0    11.0      0.004587      0     P F Yiu   
6  11499      Multigogo  5.0     1.0      0.029412      0    A S Cruz   
7  11499    Virtus Star  3.0    12.0      0.027027      0    D J Hall   
8  11499         Monica  5.0     4.0      0.161290      0  John Moore   
9  11499    Regency Gem  4.0     9.0      0.027778      0      W Y So   

           jockeyName  position positionL  ...  TR    OR           father  \
0        Joao Moreira         1       NaN  ... NaN  54.0          Duporth   
1          Zac Purton         2        nk 

In [7]:
import pandas as pd
import re

# Function to clean text columns by removing brackets and special characters
def clean_race_columns(df, columns):
    for col in columns:
        # Remove content inside parentheses and the parentheses themselves
        df[col] = df[col].apply(lambda x: re.sub(r'\(.*?\)', '', str(x)))
        
        # Remove special characters, keep only alphanumeric characters and spaces
        df[col] = df[col].str.replace(r'\W', ' ', regex=True)
        
        # Remove extra spaces
        df[col] = df[col].str.strip()
        
        # Optional: Convert to lowercase to standardize the text
        df[col] = df[col].str.lower()
        
    return df

# Clean the 'course' and 'title' columns in the 'race_data_cleaned' DataFrame
columns_to_clean = ['course', 'title']  # Update this if the column names are different in your dataset
race_data_cleaned = clean_race_columns(race_data_cleaned, columns_to_clean)

# Optionally, display the cleaned DataFrame to verify changes
print(race_data_cleaned[['course', 'title']].head())


       course                                              title
0     sha tin                                 wong leng handicap
1     sha tin                              kowloon peak handicap
2  fairyhouse  follow fairyhouse on social media beginners chase
3      exeter              tks ltd  national hunt  maiden hurdle
4     tramore     david flynn building contractors maiden hurdle


In [18]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.set_option('display.max_colwidth', None)  # Ensures full column content is displayed


In [32]:
print(race_data_cleaned.head(10))

      rid      course   time      date  \
0   11499     sha tin  06:30  19/01/01   
1   26954     sha tin  05:00  19/01/01   
2   35478  fairyhouse  02:40  19/01/01   
3    3840      exeter  01:50  19/01/01   
4  110475     tramore  12:00  19/01/01   
5  110667  cheltenham  12:50  19/01/01   
6  111356  fairyhouse  01:30  19/01/01   
7  113612     sha tin  09:10  19/01/01   
8  114194      exeter  02:25  19/01/01   
9  116279   catterick  12:25  19/01/01   

                                               title   rclass  band  ages  \
0                                 wong leng handicap      NaN   NaN     3   
1                              kowloon peak handicap      NaN   NaN     3   
2  follow fairyhouse on social media beginners chase      NaN   NaN     5   
3              tks ltd  national hunt  maiden hurdle  Class 4   NaN     4   
4     david flynn building contractors maiden hurdle      NaN   NaN     5   
5      join the betbright racing club handicap chase  Class 2  72.5     5  

In [8]:
import pandas as pd
import re

# Define cleaning functions for time and date
def normalize_time(time_str):
    if pd.isnull(time_str):
        return None
    time_parts = time_str.split(':')
    if len(time_parts) == 2:
        return int(time_parts[0]) * 60 + int(time_parts[1])  # Convert time to minutes
    elif len(time_parts) == 3:
        return int(time_parts[0]) * 3600 + int(time_parts[1]) * 60 + int(time_parts[2])  # Convert time to seconds
    return None

def clean_date(date_str):
    if pd.isnull(date_str):
        return None
    # Assuming the date format is 'YY/MM/DD'
    try:
        # Convert to datetime format for consistency
        return pd.to_datetime(date_str, format='%y/%m/%d')
    except ValueError:
        return None  # If the date is in an unexpected format, return None

# Apply cleaning functions
if 'time' in race_data_cleaned.columns:
    race_data_cleaned['time'] = race_data_cleaned['time'].apply(normalize_time)

if 'date' in race_data_cleaned.columns:
    race_data_cleaned['date'] = race_data_cleaned['date'].apply(clean_date)

# Print cleaned DataFrame with time and date columns
print("Cleaned Race Data (Time and Date):")
print(race_data_cleaned[['time', 'date']].head(10))


Cleaned Race Data (Time and Date):
   time       date
0   390 2019-01-01
1   300 2019-01-01
2   160 2019-01-01
3   110 2019-01-01
4   720 2019-01-01
5   770 2019-01-01
6    90 2019-01-01
7   550 2019-01-01
8   145 2019-01-01
9   745 2019-01-01


In [13]:
print(horse_data_cleaned.head(10))

     rid      horseName  age  saddle  decimalPrice  isFav trainerName  \
0  11499         Picken  4.0     6.0      0.312500      0      J Size   
1  11499  Noble De Love  6.0     7.0      0.333333      1     F C Lor   
2  11499    Fresh Power  5.0     3.0      0.200000      0    Y S Tsui   
3  11499     Happy Hour  4.0     5.0      0.017544      0     C H Yip   
4  11499      Super Fun  5.0     8.0      0.090909      0        L Ho   
5  11499  Blizzing Away  3.0    11.0      0.004587      0     P F Yiu   
6  11499      Multigogo  5.0     1.0      0.029412      0    A S Cruz   
7  11499    Virtus Star  3.0    12.0      0.027027      0    D J Hall   
8  11499         Monica  5.0     4.0      0.161290      0  John Moore   
9  11499    Regency Gem  4.0     9.0      0.027778      0      W Y So   

           jockeyName  position  dist  ...  TR    OR           father  \
0        Joao Moreira         1   NaN  ... NaN  54.0          Duporth   
1          Zac Purton         2   NaN  ... NaN  53

In [9]:
race_data_cleaned.to_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/races_cleaned.csv")
horse_data_cleaned.to_csv("C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_cleaned.csv")


In [28]:
import pandas as pd

# Load your dataset
file_path = "C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_cleaned.csv"
df = pd.read_csv(file_path)

# Define a list of unwanted string values
unwanted_values = ['nk', 'shd', 'hd', 'nse', 'dist', 'dht', 'nK']  # Add more as needed

# Function to clean the positionL column
def clean_positionL(value):
    if pd.isnull(value):
        return None
    value = str(value).lower().strip()
    if value.isdigit() or value.replace('.', '', 1).isdigit():
        return float(value)
    elif value in unwanted_values:
        return None
    else:
        return None

# Apply the function to the positionL column
df['positionL'] = df['positionL'].apply(clean_positionL)

# Print value counts before dropping NA values
print("Value counts before dropping NA values in positionL column:\n", df['positionL'].value_counts(dropna=False))

# Drop rows where positionL could not be converted to a number
df_cleaned = df.dropna(subset=['positionL'])

# Print value counts after dropping NA values
print("\nValue counts after dropping NA values in positionL column:\n", df_cleaned['positionL'].value_counts())



Value counts before dropping NA values in positionL column:
 positionL
NaN       101740
0.50       25959
0.75       21073
1.25       16877
1.00       15676
           ...  
108.00         1
137.00         1
111.00         1
129.00         1
154.00         1
Name: count, Length: 187, dtype: int64

Value counts after dropping NA values in positionL column:
 positionL
0.50      25959
0.75      21073
1.25      16877
1.00      15676
1.50      13075
          ...  
108.00        1
137.00        1
111.00        1
129.00        1
154.00        1
Name: count, Length: 186, dtype: int64


In [31]:

import pandas as pd
import numpy as np
from scipy.stats import zscore

# Remove duplicates
print("\nNumber of rows before removing duplicates:", len(df_cleaned))
df_cleaned = df_cleaned.drop_duplicates()
print("Number of rows after removing duplicates:", len(df_cleaned))

# Function to remove outliers using Z-score method
def remove_outliers_using_zscore(df, column, threshold=3):
    # Calculate the Z-scores for the column (including NaN values)
    z_scores = zscore(df[column], nan_policy='omit')  # Ignore NaNs during Z-score calculation
    # Identify rows where Z-score is above the threshold (both positive and negative)
    return df[np.abs(z_scores) < threshold]

# Get all numerical columns
numerical_columns = df_cleaned.select_dtypes(include=['int64', 'float64']).columns

# Handle outliers for each numerical column using Z-score method
print("\nNumber of rows before removing outliers:", len(df_cleaned))
for col in numerical_columns:
    df_cleaned = remove_outliers_using_zscore(df_cleaned, col)
print("Number of rows after removing outliers:", len(df_cleaned))

# Get unique values in the cleaned numerical columns
unique_values = {}
for col in numerical_columns:
    unique_values[col] = df_cleaned[col].unique()

# Print the unique values for each numerical column
print("\nUnique values in the cleaned numerical columns:")
for col, values in unique_values.items():
    print(f"{col}: {values[:10]}...")  # Printing first 10 unique values for brevity




Number of rows before removing duplicates: 218282
Number of rows after removing duplicates: 218282

Number of rows before removing outliers: 218282
Number of rows after removing outliers: 60420

Unique values in the cleaned numerical columns:
Unnamed: 0: [ 35  74  75  90  92  95  96 108 110 111]...
rid: [  3840 111356 114194 117432 118776 119918 127139 129290 130550 131706]...
age: [ 6.  5. 10.  8.  9.  4.  7.  3.  2.]...
saddle: [ 2.  3.  5.  1.  7. 10.  4.  9.  6.  8.]...
decimalPrice: [0.09090909 0.06666667 0.11111111 0.22222222 0.05882353 0.03846154
 0.14285714 0.07692308 0.2        0.01234568]...
isFav: [0]...
position: [ 7  4  5  2  8  3  6  9 10 11]...
positionL: [10.    5.5   2.    5.    1.   17.    2.75  3.75  3.5   1.75]...
weightSt: [11 12 10  8  9  7]...
weightLb: [ 4  6  3  7  0  1  9  2 12 11]...
RPR: [ 92. 116. 111. 135. 129.  95. 107. 123. 115. 132.]...
TR: [ 70.  80.  75. 112. 105.  84. 108. 100. 115. 110.]...
OR: [120. 117. 112. 125. 132. 119. 130. 122. 118. 135.]...

In [32]:
# Save the cleaned dataset
output_file_path = "C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_cleaned_final.csv" 
df_cleaned.to_csv(output_file_path, index=False)


In [37]:
import pandas as pd
import numpy as np

# Load your dataset
file_path = r"C:\Users\Elakkiya\Downloads\Horse Race Prediction\horses_cleaned_final.csv"
df = pd.read_csv(file_path)

# Function to handle missing values by distributing them evenly
def distribute_missing_values_evenly(df):
    for col in df.columns:
        if df[col].isnull().sum() > 0:  # Check if there are any missing values in the column
            if df[col].dtype == 'object':  # For categorical columns
                non_missing_values = df[col].dropna().values
                missing_count = df[col].isnull().sum()
                fill_values = np.random.choice(non_missing_values, size=missing_count, replace=True)
                df.loc[df[col].isnull(), col] = fill_values
            elif df[col].dtype in ['float64', 'int64']:  # For numerical columns (int and float)
                non_missing_values = df[col].dropna().values
                missing_count = df[col].isnull().sum()
                fill_values = np.random.choice(non_missing_values, size=missing_count, replace=True)
                df.loc[df[col].isnull(), col] = fill_values
    return df

# Print value counts before filling missing values
print("Value counts before filling missing values:\n")
for col in df.columns:
    if df[col].isnull().sum() > 0:
        print(f"Column: {col}")
        print(df[col].value_counts(dropna=False), "\n")

# Handle missing values
df = distribute_missing_values_evenly(df)

# Print value counts after filling missing values
print("Value counts after filling missing values:\n")
for col in df.columns:
    if df[col].isnull().sum() > 0:
        print(f"Column: {col}")
        print(df[col].value_counts(dropna=False), "\n")

# Check for any remaining missing values
print("Remaining missing values:\n", df.isnull().sum())

# Apply the function to fill missing values again to ensure it's fully filled
df = distribute_missing_values_evenly(df)


Value counts before filling missing values:

Column: dist
dist
NaN       7663
5.50      1002
4.50      1002
4.00       996
5.25       992
          ... 
100.00       1
80.00        1
96.00        1
90.00        1
111.25       1
Name: count, Length: 357, dtype: int64 

Column: headGear
headGear
NaN    33922
p       8001
t       5552
b       3955
h       2443
v       2382
tp      1869
tb       944
ht       549
tv       400
e/s      135
hp        94
eb        51
hb        49
hv        19
htb       15
he        10
et         8
htp        8
e          4
heb        3
het        2
etb        2
htv        1
hc         1
hev        1
Name: count, dtype: int64 

Value counts after filling missing values:

Remaining missing values:
 Unnamed: 0      0
rid             0
horseName       0
age             0
saddle          0
decimalPrice    0
isFav           0
trainerName     0
jockeyName      0
position        0
positionL       0
dist            0
weightSt        0
weightLb        0
headGear        

In [38]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Function for frequency encoding
def frequency_encoding(df, column):
    freq_encoding = df[column].value_counts() / len(df)
    df[column] = df[column].map(freq_encoding)
    return df

# List of categorical columns to encode
categorical_columns = ['trainerName', 'jockeyName', 'headGear', 'horseName', 'father', 'mother', 'gfather']

# Apply frequency encoding to categorical columns
for col in categorical_columns:
    df = frequency_encoding(df, col)

# Feature engineering for new features
df['win_rate'] = df['res_win'] / df['runners']
df['place_rate'] = df['res_place'] / df['runners']
df['average_margin'] = df['margin'] / df['runners']
df['weight_diff'] = df['weightSt'] - df['weightLb']

# Ensure no division by zero errors in the new features by replacing infinities with NaN
df['win_rate'] = df['win_rate'].replace([np.inf, -np.inf], np.nan)
df['place_rate'] = df['place_rate'].replace([np.inf, -np.inf], np.nan)
df['average_margin'] = df['average_margin'].replace([np.inf, -np.inf], np.nan)

# Convert relevant columns to numeric
df['dist'] = pd.to_numeric(df['dist'], errors='coerce')

# Check for any remaining missing values
print("Remaining missing values:\n", df.isnull().sum())

# Apply scaling for numerical columns
numerical_columns = ['win_rate', 'place_rate', 'weightSt', 'average_margin', 'weight_diff', 'weightLb','age', 'position', 'positionL', 'decimalPrice', 'saddle', 
                     'dist', 'isFav', 'RPR', 'TR', 'OR', 'runners', 'margin',	'weight', 'res_win', 'res_place']

# Initialize StandardScaler
scaler = StandardScaler()

# Scale the numerical columns
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Print the dataframe to check the new features, encoded columns, and scaled columns
print(df.head())

# Save the cleaned, engineered, and scaled dataset
output_file_path = "C:/Users/Elakkiya/Downloads/Horse Race Prediction/horses_engineered_scaled.csv"
df.to_csv(output_file_path, index=False)


Remaining missing values:
 Unnamed: 0        0
rid               0
horseName         0
age               0
saddle            0
decimalPrice      0
isFav             0
trainerName       0
jockeyName        0
position          0
positionL         0
dist              0
weightSt          0
weightLb          0
headGear          0
RPR               0
TR                0
OR                0
father            0
mother            0
gfather           0
runners           0
margin            0
weight            0
res_win           0
res_place         0
win_rate          0
place_rate        0
average_margin    0
weight_diff       0
dtype: int64
   Unnamed: 0     rid  horseName       age    saddle  decimalPrice  isFav  \
0          35    3840   0.000050  0.359139 -1.115340     -0.142190    0.0   
1          74  111356   0.000033 -0.127957 -1.115340     -0.506697    0.0   
2          75  111356   0.000050  2.307523 -0.852229      0.161566    0.0   
3          90  114194   0.000099  1.333331 -0.326006